In [23]:
import pandas as pd
import numpy as np
import warnings as wn
import datetime as dt
import xgboost as xgb

wn.simplefilter("ignore")

In [24]:
dtypes_auc = {"ref_type_id": np.int8, "source_id": np.int16, "device_id": np.int64 }

dtypes_ins = { "application_id": np.int16, "ref_type":"category", "click_hash":"category", "device_model":"category",
               "device_countrycode":"category", "device_brand": "category", "session_user_agent":"category",
               "kind":"category", "wifi":"category", "device_language":"category"}      

dtypes_clk = { "advertiser_id": np.int8, "action_id": np.float16, "source_id": np.int8, "country_code": "category", 
               "latitude": np.float16, "longitude": np.float16, "wifi_connection": "category", "carrier_id": np.float16,
               "trans_id": "object", "os_major":np.float32, "brand": np.float16, "touchX": np.float16,
               "touchY": np.float16  }

dtypes_evt = { "event_id":np.int32, "application_id": np.int32, "attributed":"category", "device_countrycode":"category",
               "device_os_version":"category", "device_brand": np.float32, "device_model": "category",
               "device_city":"category", "session_user_agent":"category", "user_agent":np.float32, "carrier":"category",
               "kind":"category", "wifi":"category", "connection_type":"category", "device_language":"category",
               "trans_id":"object" }

In [25]:
loc = r"D:\FacundoTorraca\Documents\Datos_ML_TP2\CSV"

In [26]:
auc = pd.read_csv( loc + "\\auctions.csv.gzip", compression = "gzip", dtype = dtypes_auc ) 
auc["date"] = pd.to_datetime( auc["date"], errors = "coerce" )

In [37]:
ins = pd.read_csv( loc + '\\installs.csv.gzip', compression = 'gzip', dtype = dtypes_ins )
ins["created"] = pd.to_datetime( ins["created"], errors = "coerce" )
ins = ins.rename( columns = {"created":"date"} )

In [28]:
evt = pd.read_csv( loc + '\\events.csv.gzip', compression = "gzip", dtype = dtypes_evt )
evt["date"] = pd.to_datetime( evt["date"], errors = "coerce" )

In [38]:
clk = pd.read_csv( loc + '\\clicks.csv.gzip', compression = "gzip" )
clk["touchX"] = clk["touchX"].fillna(0)
clk["touchY"] = clk["touchY"].fillna(0)
clk["touchX"] = pd.to_numeric( clk["touchX"], downcast = "float", errors = "coerce" )
clk["touchY"] = pd.to_numeric( clk["touchY"], downcast = "float", errors = "coerce" )
clk["created"] = pd.to_datetime( clk["created"], errors = "coerce" )
clk = clk.rename( columns = {"created":"date"} )

In [17]:
tar = pd.read_csv( r"C:\Users\FacundoTorraca\Documents\Facultad\Organizacion de Datos\Trabajo Practico 2\CSV\target_final_competencia_revamped.csv" )

### Acomodo los FEATURES para cada categoria

In [39]:
ins_1 = ins.loc[ ins['date'].dt.day.between(18,20) ]; ins_1["ventana"] = "18_20";
ins_2 = ins.loc[ ins['date'].dt.day.between(19,21) ]; ins_2["ventana"] = "19_21";
ins_3 = ins.loc[ ins['date'].dt.day.between(20,22) ]; ins_3["ventana"] = "20_22";
ins_4 = ins.loc[ ins['date'].dt.day.between(21,23) ]; ins_4["ventana"] = "21_23";
ins_5 = ins.loc[ ins['date'].dt.day.between(22,24) ]; ins_5["ventana"] = "22_24";
ins_6 = ins.loc[ ins['date'].dt.day.between(23,25) ]; ins_6["ventana"] = "23_25";
ins_7 = ins.loc[ ins['date'].dt.day.between(24,26) ]; ins_7["ventana"] = "24_26";

In [40]:
clk_1 = clk.loc[ clk['date'].dt.day.between(18,20) ]; clk_1["ventana"] = "18_20";
clk_2 = clk.loc[ clk['date'].dt.day.between(19,21) ]; clk_2["ventana"] = "19_21";
clk_3 = clk.loc[ clk['date'].dt.day.between(20,22) ]; clk_3["ventana"] = "20_22";
clk_4 = clk.loc[ clk['date'].dt.day.between(21,23) ]; clk_4["ventana"] = "21_23";
clk_5 = clk.loc[ clk['date'].dt.day.between(22,24) ]; clk_5["ventana"] = "22_24";
clk_6 = clk.loc[ clk['date'].dt.day.between(23,25) ]; clk_6["ventana"] = "23_25";
clk_7 = clk.loc[ clk['date'].dt.day.between(24,26) ]; clk_7["ventana"] = "24_26";

In [34]:
evt_1 = evt.loc[ evt['date'].dt.day.between(18,20) ]; evt_1["ventana"] = "18_20"; 
evt_2 = evt.loc[ evt['date'].dt.day.between(19,21) ]; evt_2["ventana"] = "19_21";
evt_3 = evt.loc[ evt['date'].dt.day.between(20,22) ]; evt_3["ventana"] = "20_22";
evt_4 = evt.loc[ evt['date'].dt.day.between(21,23) ]; evt_4["ventana"] = "21_23";
evt_5 = evt.loc[ evt['date'].dt.day.between(22,24) ]; evt_5["ventana"] = "22_24";
evt_6 = evt.loc[ evt['date'].dt.day.between(23,25) ]; evt_6["ventana"] = "23_25";
evt_7 = evt.loc[ evt['date'].dt.day.between(24,26) ]; evt_7["ventana"] = "24_26";

In [35]:
auc_1 = auc.loc[ auc['date'].dt.day.between(18,20) ]; auc_1["ventana"] = "18_20";
auc_2 = auc.loc[ auc['date'].dt.day.between(19,21) ]; auc_2["ventana"] = "19_21";
auc_3 = auc.loc[ auc['date'].dt.day.between(20,22) ]; auc_3["ventana"] = "20_22";
auc_4 = auc.loc[ auc['date'].dt.day.between(21,23) ]; auc_4["ventana"] = "21_23";
auc_5 = auc.loc[ auc['date'].dt.day.between(22,24) ]; auc_5["ventana"] = "22_24";
auc_6 = auc.loc[ auc['date'].dt.day.between(23,25) ]; auc_6["ventana"] = "23_25";
auc_7 = auc.loc[ auc['date'].dt.day.between(24,26) ]; auc_7["ventana"] = "24_26";

### Acomodo Features de INSTALLS & AUCTIONS para prediccion

In [41]:
ins_1_pre = ins_1[ ["ref_hash", "date"] ]; ins_1_pre["_sc"] = ins_1_pre["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); ins_1_pre["_sc"] = ins_1_pre["_sc"].dt.total_seconds(); del( ins_1_pre["date"] )
ins_2_pre = ins_2[ ["ref_hash", "date"] ]; ins_2_pre["_sc"] = ins_2_pre["date"] - dt.datetime( year = 2019, month = 4, day = 19 ); ins_2_pre["_sc"] = ins_2_pre["_sc"].dt.total_seconds(); del( ins_2_pre["date"] )
ins_3_pre = ins_3[ ["ref_hash", "date"] ]; ins_3_pre["_sc"] = ins_3_pre["date"] - dt.datetime( year = 2019, month = 4, day = 20 ); ins_3_pre["_sc"] = ins_3_pre["_sc"].dt.total_seconds(); del( ins_3_pre["date"] )
ins_4_pre = ins_4[ ["ref_hash", "date"] ]; ins_4_pre["_sc"] = ins_4_pre["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); ins_4_pre["_sc"] = ins_4_pre["_sc"].dt.total_seconds(); del( ins_4_pre["date"] )
ins_5_pre = ins_5[ ["ref_hash", "date"] ]; ins_5_pre["_sc"] = ins_5_pre["date"] - dt.datetime( year = 2019, month = 4, day = 22 ); ins_5_pre["_sc"] = ins_5_pre["_sc"].dt.total_seconds(); del( ins_5_pre["date"] )
ins_6_pre = ins_6[ ["ref_hash", "date"] ]; ins_6_pre["_sc"] = ins_6_pre["date"] - dt.datetime( year = 2019, month = 4, day = 23 ); ins_6_pre["_sc"] = ins_6_pre["_sc"].dt.total_seconds(); del( ins_6_pre["date"] )
ins_7_pre = ins_7[ ["ref_hash", "date"] ]; ins_7_pre["_sc"] = ins_7_pre["date"] - dt.datetime( year = 2019, month = 4, day = 24 ); ins_7_pre["_sc"] = ins_7_pre["_sc"].dt.total_seconds(); del( ins_7_pre["date"] )

In [42]:
auc_1_pre = auc_1[ ["device_id", "date"] ]; auc_1_pre["_st"] = auc_1_pre["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); auc_1_pre["_st"] = auc_1_pre["_st"].dt.total_seconds(); del( auc_1_pre["date"] )
auc_2_pre = auc_2[ ["device_id", "date"] ]; auc_2_pre["_st"] = auc_2_pre["date"] - dt.datetime( year = 2019, month = 4, day = 19 ); auc_2_pre["_st"] = auc_2_pre["_st"].dt.total_seconds(); del( auc_2_pre["date"] )
auc_3_pre = auc_3[ ["device_id", "date"] ]; auc_3_pre["_st"] = auc_3_pre["date"] - dt.datetime( year = 2019, month = 4, day = 20 ); auc_3_pre["_st"] = auc_3_pre["_st"].dt.total_seconds(); del( auc_3_pre["date"] )
auc_4_pre = auc_4[ ["device_id", "date"] ]; auc_4_pre["_st"] = auc_4_pre["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); auc_4_pre["_st"] = auc_4_pre["_st"].dt.total_seconds(); del( auc_4_pre["date"] )
auc_5_pre = auc_5[ ["device_id", "date"] ]; auc_5_pre["_st"] = auc_5_pre["date"] - dt.datetime( year = 2019, month = 4, day = 22 ); auc_5_pre["_st"] = auc_5_pre["_st"].dt.total_seconds(); del( auc_5_pre["date"] )
auc_6_pre = auc_6[ ["device_id", "date"] ]; auc_6_pre["_st"] = auc_6_pre["date"] - dt.datetime( year = 2019, month = 4, day = 23 ); auc_6_pre["_st"] = auc_6_pre["_st"].dt.total_seconds(); del( auc_6_pre["date"] )
auc_7_pre = auc_7[ ["device_id", "date"] ]; auc_7_pre["_st"] = auc_7_pre["date"] - dt.datetime( year = 2019, month = 4, day = 24 ); auc_7_pre["_st"] = auc_7_pre["_st"].dt.total_seconds(); del( auc_7_pre["date"] )

### Agregamos los LABEL´S

In [44]:
ins_1["_sc"] = ins_1_pre["_sc"]; auc_1["_st"] = auc_1_pre["_st"]
ins_2["_sc"] = ins_2_pre["_sc"]; auc_2["_st"] = auc_2_pre["_st"]
ins_3["_sc"] = ins_3_pre["_sc"]; auc_3["_st"] = auc_3_pre["_st"]
ins_4["_sc"] = ins_4_pre["_sc"]; auc_4["_st"] = auc_4_pre["_st"]
ins_5["_sc"] = ins_5_pre["_sc"]; auc_5["_st"] = auc_5_pre["_st"]
ins_6["_sc"] = ins_6_pre["_sc"]; auc_6["_st"] = auc_6_pre["_st"]
ins_7["_sc"] = ins_7_pre["_sc"]; auc_7["_st"] = auc_7_pre["_st"]

### Los guardamos en CSV

In [46]:
loc_to_save = r"C:\Users\FacundoTorraca\Documents\Facultad\Organizacion de Datos\Trabajo Practico 2\Sets de Entrenamiento"

In [47]:
ins_1.to_csv( loc_to_save + "\\ins_1.csv", index = False ); clk_1.to_csv( loc_to_save + "\\clk_1.csv", index = False ); 
ins_2.to_csv( loc_to_save + "\\ins_2.csv", index = False ); clk_2.to_csv( loc_to_save + "\\clk_2.csv", index = False );
ins_3.to_csv( loc_to_save + "\\ins_3.csv", index = False ); clk_3.to_csv( loc_to_save + "\\clk_3.csv", index = False );
ins_4.to_csv( loc_to_save + "\\ins_4.csv", index = False ); clk_4.to_csv( loc_to_save + "\\clk_4.csv", index = False ); 
ins_5.to_csv( loc_to_save + "\\ins_5.csv", index = False ); clk_5.to_csv( loc_to_save + "\\clk_5.csv", index = False );
ins_6.to_csv( loc_to_save + "\\ins_6.csv", index = False ); clk_6.to_csv( loc_to_save + "\\clk_6.csv", index = False ); 
ins_7.to_csv( loc_to_save + "\\ins_7.csv", index = False ); clk_7.to_csv( loc_to_save + "\\clk_7.csv", index = False ); 

In [48]:
auc_1.to_csv( loc_to_save + "\\auc_1.csv", index = False ); evt_1.to_csv( loc_to_save + "\\evt_1.csv", index = False ); 
auc_2.to_csv( loc_to_save + "\\auc_2.csv", index = False ); evt_2.to_csv( loc_to_save + "\\evt_2.csv", index = False );
auc_3.to_csv( loc_to_save + "\\auc_3.csv", index = False ); evt_3.to_csv( loc_to_save + "\\evt_3.csv", index = False ); 
auc_4.to_csv( loc_to_save + "\\auc_4.csv", index = False ); evt_4.to_csv( loc_to_save + "\\evt_4.csv", index = False ); 
auc_5.to_csv( loc_to_save + "\\auc_5.csv", index = False ); evt_5.to_csv( loc_to_save + "\\evt_5.csv", index = False );
auc_6.to_csv( loc_to_save + "\\auc_6.csv", index = False ); evt_6.to_csv( loc_to_save + "\\evt_6.csv", index = False );
auc_7.to_csv( loc_to_save + "\\auc_7.csv", index = False ); evt_7.to_csv( loc_to_save + "\\evt_7.csv", index = False ); 